<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4: West Nile Virus Prediction

Note: This is part 2 of the code notebook covering the following:-

1. [Library Imports & Functions Creation](#1.-Library-Imports-&-Functions-Creation)
2. [Modelling Using Pycaret](#2.-Modelling-Using-Pycaret)
3. [Model Evaluation](#3.-Model-Evaluation)
4. [Cost-Benefit Analysis](#4.-Cost-Benefit-Analysis-of-Spraying)
5. [Conclusion](#5.-Conclusion)

In this notebook, we will continue to build on from our data cleaning, feature selection & engineering and EDA in part 1 in an attempt to create a model that will help classify and predict areas with the West Nile Virus. 

## Reiteration of Problem Statement

This project aims to predict West Nile Virus (WNV) in mosquitoes across the city of Chicago for the missing years 2008, 2010, 2012 and 2014. It will make use of the dataset provided by [Kaggle](https://www.kaggle.com/competitions/predict-west-nile-virus/data) consisting of years 2007, 2009, 2011 and 2013.

In summary, the project shall :-
1. perform data cleaning on the datasets provided;
2. provide exploratory data analysis (EDA) on the datasets;
3. merge the datasets for further analysis;
4. produce classification models to predict the missing years;
5. identify model with the best AUC score through extensive hyperparameter tuning; and
6. present a cost-benefit analysis on the usage of pesticides to control the population of mosquitoes.

## 1. Library Imports & Functions Creation

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics as metrics
from pycaret.classification import *

In [3]:
#making function to change the score and making appropriate dataframe for Kaggle submission
def kaggle_submission(preds, model_name):
    
    #changing the scores
    preds['Prob'] = preds['Score']
    preds.loc[preds['Label'] == 0, 'Prob'] = 1-preds['Prob']
    
    #making appropriate dataframe for submission
    submission = pd.merge(test['id'], preds['Prob'], left_index = True, right_index = True)
    
    submission.rename({'id' : 'Id',
                      'Prob' : 'WnvPresent'},
                     inplace = True,
                     axis = 1)
    
    #export
    submission.to_csv(f'../datasets/submission_{model_name}.csv', index=False)
    
    return submission

### 1.1 Data Importing

In [4]:
#import datasets
train = pd.read_csv('../datasets/train_final.csv')
test= pd.read_csv('../datasets/test_final.csv')

In [5]:
#check train dataset
print(train.shape)
train.head()

(10400, 27)


,latitude,longitude,wnvpresent,year,weekofyear,hot_spots,tmax,tmin,tavg,dewpoint,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,relativehumidity,month_5,month_6,month_7,month_8,month_9,month_10,species_culex pipiens,species_culex pipiens/restuans,species_culex restuans,species_others
0,41.921965,-87.632085,0,2007,22,1,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921321,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,41.731922,-87.677512,0,2007,22,1,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921321,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,41.731922,-87.677512,0,2007,22,1,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921321,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,41.720848,-87.666014,0,2007,22,1,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921321,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,41.862292,-87.648860,0,2007,22,1,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921321,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
#check test dataset
print(test.shape)
test

(116293, 27)


,id,latitude,longitude,year,weekofyear,hot_spots,tmax,tmin,tavg,dewpoint,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,relativehumidity,month_5,month_6,month_7,month_8,month_9,month_10,species_culex pipiens,species_culex pipiens/restuans,species_culex restuans,species_others
0,1,41.954690,-87.800991,2008,24,1,85.5,58.5,72.0,52.5,0.0,29.185,29.88,1.8,10.5,5.25,50.247640,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,41.954690,-87.800991,2008,24,1,85.5,58.5,72.0,52.5,0.0,29.185,29.88,1.8,10.5,5.25,50.247640,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,41.954690,-87.800991,2008,24,1,85.5,58.5,72.0,52.5,0.0,29.185,29.88,1.8,10.5,5.25,50.247640,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,41.954690,-87.800991,2008,24,1,85.5,58.5,72.0,52.5,0.0,29.185,29.88,1.8,10.5,5.25,50.247640,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,41.954690,-87.800991,2008,24,1,85.5,58.5,72.0,52.5,0.0,29.185,29.88,1.8,10.5,5.25,50.247640,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116288,116289,41.925652,-87.633590,2014,40,1,75.0,50.5,63.0,47.0,0.0,29.605,30.30,2.6,18.5,5.25,55.850352,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
116289,116290,41.925652,-87.633590,2014,40,1,75.0,50.5,63.0,47.0,0.0,29.605,30.30,2.6,18.5,5.25,55.850352,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
116290,116291,41.925652,-87.633590,2014,40,1,75.0,50.5,63.0,47.0,0.0,29.605,30.30,2.6,18.5,5.25,55.850352,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
116291,116292,41.925652,-87.633590,2014,40,1,75.0,50.5,63.0,47.0,0.0,29.605,30.30,2.6,18.5,5.25,55.850352,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [7]:
train['wnvpresent'].value_counts(normalize=True)

0    0.947115
1    0.052885
Name: wnvpresent, dtype: float64

Given that the train dataset is imbalanced at only 5% with WNV Present detected, the usage of accuracy as a metric for our model would be flawed. Our metric of choice would be AUC to optimize our model on.

## 2. Modelling Using Pycaret

### 2.1 Setting Up Environment

First we use the `setup()` function to initialize the environment in pycaret and create the transformation pipeline to prepare the data for modeling and deployment. It takes two mandatory parameters: our train dataset and target column (our y variable) which is `'wnvpresent'`. 

When `setup()` is executed, PyCaret's inference algorithm will automatically infer the data types for all features based on certain properties and display a table containing the features and their inferred data types after setup() is executed.

Our train and test datasets have gone through `OneHotEncoding` for the categorical features in the 1st notebook. Pycaret is able to scale our data using the `normalize=True` parameter and we will be using z-score which is the same as `StandardScaler` (or Z-score normalization). Continuous features like `'relativehumidity'` has a wide range of values and due to differences in the magnitude, our model may neglect units of higher/lower numbers. 

Pycaret is able to detect and classify the features of the dataset. For example, it was able to detect `'year'` as the categorical feature. However, the features that `OneHotEncoding` had transformed was detected to be of also categorical in nature instead. Furthermore, Pycaret will do categorical data encoding if feature is classified as such. To bypass this, we had to list out these features to override the classification before running the dataset through Pycaret environment.

In addition, since the dataset in class imbalance as mentioned earlier, we set the `fix_imbalance=True` parameter in which Pycaret will help alleviate this problem by using Synthetic Minority OverSampling Technique (SMOTE).


In [8]:
pce_1 = setup(data = train, target = 'wnvpresent', session_id = 5, 
              data_split_stratify=True, 
              normalize=True, normalize_method='zscore', #to scale our data
              numeric_features = ['year','weekofyear','hot_spots','month_5', 'month_6','month_7','month_8','month_9','month_10',
                                  'species_culex pipiens','species_culex pipiens/restuans','species_culex restuans','species_others'
                                 ], #since the categorical features has been converted, it ought to be treated as numerical features
              fix_imbalance=True, #pycaret to use default SMOTE to fix the imbalance dataset 
             )

,Description,Value
0,session_id,5
1,Target,wnvpresent
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(10400, 27)"
5,Missing Values,False
6,Numeric Features,26
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
%time compare_models(n_select=2, sort = 'AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9026,0.8406,0.3738,0.2344,0.2861,0.2370,0.2455,0.1450
gbc,Gradient Boosting Classifier,0.8281,0.8385,0.5970,0.1739,0.2685,0.2034,0.2543,0.8380
ada,Ada Boost Classifier,0.8060,0.8314,0.6391,0.1620,0.2580,0.1897,0.2502,0.2290
xgboost,Extreme Gradient Boosting,0.9141,0.8311,0.2755,0.2335,0.2511,0.2062,0.2078,0.9920
lr,Logistic Regression,0.7170,0.8268,0.7948,0.1346,0.2300,0.1534,0.2454,0.7290
lda,Linear Discriminant Analysis,0.6936,0.8262,0.8183,0.1279,0.2210,0.1426,0.2392,0.0470
nb,Naive Bayes,0.2590,0.8037,0.9949,0.0663,0.1244,0.0280,0.1175,0.0190
rf,Random Forest Classifier,0.8986,0.7925,0.3376,0.2107,0.2579,0.2068,0.2143,0.4690
et,Extra Trees Classifier,0.8997,0.7508,0.3246,0.2085,0.2527,0.2018,0.2081,0.3930
knn,K Neighbors Classifier,0.8571,0.7242,0.4462,0.1736,0.2493,0.1874,0.2132,0.3910


CPU times: user 10.9 s, sys: 604 ms, total: 11.5 s
Wall time: 51.8 s


[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                random_state=5, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
            

Once the environment was setup, we were able to generate models based on default parameters in the Pycaret library. The `compare_models` function trains all models in the model library and scores them using stratified cross validation for metric evaluation. The output prints a score grid that shows average Accuracy, AUC, Recall, Precision, F1, Kappa, and MCC across the folds (10 by default) along with training times. The score grid with the highest performing metrics are highlighted accordingly. The grid is sorted using 'AUC' (highest to lowest), as the Kaggle competition will be looking at ROC-AUC scores for the submission, by passing in the `optimize='AUC'` parameter. Therefore, we will be looking to optimize AUC as much as possible, while maintaining a good score in Accuracy and Recall.

The `compare_models` also returned the best performing models based on AUC scores with the `n_select=2` parameter. The top 2 highest performing models with respect to Accuracy, AUC and Recall scores are **Gradient Boosting Classifier** and **AdaBoost Classifier**. Although Light Gradient Boosting Machine has the best AUC score of 0.84 and has Accuracy of 0.90, it's Recall score of 0.37 is far from good to be chosen as a model to generate. In contrast, the 2 models mentioned has a good balance of scores in the 3 targeted scoring metrics for our potential final model.

After narrowing down to these two models, we will attempt to increase its performance by tuning its hyperparameters. Pycaret also has the function to optimize the models generated by using the `tune_model` that helps tune the hyperparameters of the models.

### 2.2 Baseline Model (Dummy Classifier)

First, we have to generate a baseline model as a comparison to generate the scores and for our models to beat. We are using Dummy Classifier as our baseline model. It is a simple classifier to use as baseline comparison as it makes predictions that ignore the input features. 

In [10]:
#create dummy model based on default params
%time dummy_model = create_model('dummy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9478,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.9478,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.9478,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.9478,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.9464,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.9464,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.9464,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.9464,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.9464,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000


CPU times: user 348 ms, sys: 29.8 ms, total: 378 ms
Wall time: 525 ms


In [11]:
#test scores based on default param
pred_dummy = predict_model(dummy_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Dummy Classifier,0.9471,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000


| Model (parameter) | Train Accuracy Score | Test Accuracy Score | Train AUC | Test AUC | Train Recall | Test Recall | Kaggle Score (Public) |
|---|---|---|---|---|---|---|---|
| Baseline (default)  | 0.9471 | 0.9471 | 0.5000 | 0.5000 | 0.0000 | 0.0000 | - |

Our baseline model scores are as such and we will be using these scores as a benchmark for our models to beat. The more important scoring metrics to concentrate is AUC and Recall. Keep in mind that dummy classifier has a high accuracy score is because our dataset has an imbalance of 0 and 1 class and therefore, the 'prediction' is made up of the true negative since Dummy Classifier does not take other features as inputs.

### 2.3 Gradient Boosting Classifier

In [12]:
#create gbc model based on default params
%time gbc_model = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8365,0.8762,0.6842,0.1955,0.3041,0.2426,0.3046
1,0.8242,0.8098,0.5526,0.1591,0.2471,0.1806,0.2262
2,0.8434,0.8856,0.6579,0.1984,0.3049,0.2443,0.3007
3,0.8091,0.7800,0.5000,0.1367,0.2147,0.1446,0.1845
4,0.8146,0.8443,0.6154,0.1667,0.2623,0.1944,0.2494
5,0.8077,0.8838,0.7949,0.1902,0.3069,0.2413,0.3259
6,0.8338,0.8689,0.6923,0.1985,0.3086,0.2458,0.3086
7,0.8049,0.7981,0.4615,0.1295,0.2022,0.1294,0.1638
8,0.8407,0.8195,0.5641,0.1818,0.2750,0.2111,0.2543


CPU times: user 3.31 s, sys: 69.5 ms, total: 3.37 s
Wall time: 11.5 s


In [13]:
#test scores based on default param
pred_gbc = predict_model(gbc_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8315,0.8474,0.6788,0.1915,0.2987,0.2356,0.2975


In [14]:
#hyperparameter tuning 
%time tuned_gbc = tune_model(gbc_model, n_iter = 50, optimize='AUC', tuner_verbose=False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8297,0.8925,0.7368,0.1972,0.3111,0.2493,0.3209
1,0.8091,0.8245,0.5789,0.1517,0.2404,0.1719,0.2232
2,0.8420,0.8812,0.6316,0.1920,0.2945,0.2331,0.2862
3,0.7926,0.7749,0.5000,0.1258,0.2011,0.1284,0.1693
4,0.8187,0.8527,0.6667,0.1793,0.2826,0.2165,0.2785
5,0.7940,0.9001,0.8205,0.1829,0.2991,0.2318,0.3230
6,0.8255,0.8660,0.7436,0.1986,0.3135,0.2501,0.3227
7,0.7995,0.8007,0.5385,0.1409,0.2234,0.1513,0.1968
8,0.8269,0.8271,0.6154,0.1778,0.2759,0.2102,0.2632


CPU times: user 6.64 s, sys: 370 ms, total: 7.01 s
Wall time: 4min 20s


In [15]:
#showing the best params
print(tuned_gbc)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=6,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.02, min_impurity_split=None,
                           min_samples_leaf=3, min_samples_split=7,
                           min_weight_fraction_leaf=0.0, n_estimators=280,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=5, subsample=0.8, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [16]:
#generate interface that will display various plots
evaluate_model(tuned_gbc)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [17]:
#test scores based on tuned model
pred_holdout_gbc = predict_model(tuned_gbc)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8174,0.8481,0.7030,0.1821,0.2893,0.2241,0.2925


Once the model has been tuned, Pycaret requires us to use the `finalize_model` function. What this does is that it will train the final model by fitting it onto the complete dataset including the test/hold-out sample (30% in this case) that was used in the setup of Pycaret environment. The purpose of this function is to train the model on the complete dataset before it is deployed in production. Following which, the model is fed the unseen test dataset for the Kaggle submission.

The output of the `predict_model` will give us two new columns; `Label` and `Score`. 
* `Label` column will show the predicted classification (0 being absence of WNV and 1 being presence of WNV) by the model.
* `Score` column will show the probability that the Label is of the positive class.  

Therefore an additional step is required before making the submission to Kaggle which is to inverse the Score by subracting 1 to get the probability that the Label is of the negative class (done in our function when creating the Kaggle dataset submission). **This is only done for rows that are predicted to be 0 in the Label column.** Following which, the new dataframe was created to show only `Id` and `WnvPresent` for Kaggle scoring and submission.

In [18]:
#finalizing the model by training it on the complete dataset before it is deployed for predicting the test datasest
final_gbc = finalize_model(tuned_gbc)

In [19]:
#predicting on test dataset
pred_unseen_gbc = predict_model(final_gbc, data = test)

In [20]:
#making dataset for kaggle submission format using the function created
kaggle_submission(pred_unseen_gbc, 'gbc')

,Id,WnvPresent
0,1,0.1302
1,2,0.1008
2,3,0.1215
3,4,0.0910
4,5,0.0910
...,...,...
116288,116289,0.2195
116289,116290,0.2195
116290,116291,0.2195
116291,116292,0.2886


We analyzed the performance of the Gradient Boosting Classifier (GBC) model by using the `evaluate_model()` function which displays a user interactive interface for all of the available plots for a given model. It internally uses the plot_model() function. The main focus in this portion is to evaluate the model based on the ROC-AUC score and graph. With an AUC score of 0.85, it suggests the 'west nile virus' and 'no west nile virus' populations are well separated, and the model is nearly as good as it can get. It also shows that the model is better at predicting 'no west nile virus' populations at at lower decision threshold, with the ROC for class 0 being higher and steeper up to around 0.1 False Positive Rate.

From the markdown table below, we see the improvement in the model for AUC and Recall after the hyperparameter tuning as well:

| Model (parameter) | Train Accuracy Score | Test Accuracy Score | Train AUC | Test AUC | Train Recall | Test Recall | Kaggle Score (Public) |
|---|---|---|---|---|---|---|---|
| Baseline (default)  | 0.9471 | 0.9471 | 0.5000 | 0.5000 | 0.0000 | 0.0000 | - |
| Gradient Boosting Classifier (default) | 0.8281 | 0.8315 | 0.8385 | 0.8474 | 0.5970 | 0.6788 | - |
| Gradient Boosting Classifier (tuned) | 0.8195 | 0.8174 | 0.8456 | 0.8481 | 0.6332 | 0.7030 | 0.7742 |

After our submission to Kaggle with the predictions from the tuned GBC model, the final public score was **0.7742**.

### 2.4 AdaBoost Classifier

In [21]:
#create lgbm model based on default param
ada_model = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8077,0.8605,0.7368,0.1772,0.2857,0.2201,0.2959
1,0.8077,0.8240,0.6842,0.1688,0.2708,0.2042,0.2716
2,0.8022,0.8650,0.7368,0.1728,0.2800,0.2135,0.2902
3,0.8063,0.7799,0.5263,0.1399,0.2210,0.1510,0.1949
4,0.7885,0.8386,0.6154,0.1472,0.2376,0.1655,0.2234
5,0.7940,0.8818,0.8462,0.1864,0.3056,0.2387,0.3344
6,0.8214,0.8436,0.6923,0.1862,0.2935,0.2283,0.2938
7,0.7734,0.7856,0.5128,0.1205,0.1951,0.1187,0.1615
8,0.8146,0.8044,0.4872,0.1418,0.2197,0.1490,0.1861


In [22]:
#test scores based on default param
pred_ada = predict_model(ada_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ada Boost Classifier,0.8026,0.8331,0.7152,0.1718,0.2770,0.2096,0.2823


In [23]:
#tuning lgbm model
%time tuned_ada = tune_model(ada_model, n_iter = 50, optimize='AUC', tuner_verbose=False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8255,0.8640,0.7895,0.2013,0.3209,0.2592,0.3402
1,0.8187,0.8191,0.6579,0.1736,0.2747,0.2094,0.2711
2,0.8324,0.8769,0.7105,0.1957,0.3068,0.2450,0.3119
3,0.7857,0.7921,0.5263,0.1266,0.2041,0.1309,0.1761
4,0.8022,0.8512,0.6154,0.1569,0.2500,0.1800,0.2366
5,0.8036,0.8882,0.8462,0.1941,0.3158,0.2505,0.3445
6,0.8365,0.8580,0.7436,0.2101,0.3277,0.2664,0.3363
7,0.7816,0.7869,0.5385,0.1296,0.2090,0.1342,0.1807
8,0.8077,0.8190,0.5641,0.1517,0.2391,0.1690,0.2174


CPU times: user 5.19 s, sys: 517 ms, total: 5.71 s
Wall time: 5min 34s


In [24]:
#showing the best params
print(tuned_ada)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.5,
                   n_estimators=170, random_state=5)


In [25]:
#generate interface that will display various plots
evaluate_model(tuned_ada)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [26]:
#test scores based on tuned model
pred_holdout_ada = predict_model(tuned_ada)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ada Boost Classifier,0.8158,0.8391,0.7212,0.1836,0.2927,0.2277,0.2992


In [27]:
#finalizing the model by training it on the complete dataset before it is deployed for predicting the test datasest
final_ada = finalize_model(tuned_ada)

In [28]:
#predicting on test dataset
pred_unseen_ada = predict_model(final_ada, data = test)

In [30]:
kaggle_submission(pred_unseen_ada, 'ada')

,Id,WnvPresent
0,1,0.4324
1,2,0.4300
2,3,0.4327
3,4,0.3816
4,5,0.3816
...,...,...
116288,116289,0.4416
116289,116290,0.4416
116290,116291,0.4416
116291,116292,0.4944


After creating and tuning a model for AdaBoost Classifier (ADA), we see that the model had improved in all scoring metrics after hyperparameter tuning by Pycaret. Furthermore, the scores in both train and holdout test score across the scoring metrics is better than GBC. 

Below table summarised the final scores for the models for comparison:

| Model (parameter) | Train Accuracy Score | Test Accuracy Score | Train AUC | Test AUC | Train Recall | Test Recall | Kaggle Score (Public) |
|---|---|---|---|---|---|---|---|
| Baseline (default)  | 0.9471 | 0.9471 | 0.5000 | 0.5000 | 0.0000 | 0.0000 | - |
| Gradient Boosting Classifier (default) | 0.8281 | 0.8315 | 0.8385 | 0.8474 | 0.5970 | 0.6788 | - |
| Gradient Boosting Classifier (tuned) | 0.8195 | 0.8174 | 0.8456 | 0.8481 | 0.6332 | 0.7030 | 0.7742 |
| AdaBoost Classifier (default) | 0.8060 | 0.8026 | 0.8314 | 0.8331 | 0.6391 | 0.7152 | - |
| AdaBoost Classifier (tuned) | 0.8158 | 0.8158 | 0.8397 | 0.8391 | 0.6571 | 0.7212 | 0.7793 |

After our submission to Kaggle with the predictions from the tuned ADA model, the final public score was **0.7793**.
Thus, since the ADA model bested both the baseline and GBC model in all scoring metrics, we have chose the AdaBoost Classifier as the final model for our project.

## 3. Model Evaluation

We have found that the tuned ADA model gave us the best results from the scoring metrics. It is important for our project that the model be able to perform based on AUC score for the Kaggle submission and on Recall score for the business aspect. We would want to reduce predicting False Negatives as our model aims to predict areas or hotspots that have WNV present. It would be detrimental, money-wise, to spray areas that are predicted to have WNV present but is actually no WNV present (False Negative). Here we will be looking at the confusion matrix and ROC-AUC score to evaluate this. We will also be looking at the features that the model deemed to be important in its prediction.

### 3.1 ROC-AUC and Recall

![](../imgs/confusion-matrix.png)

![](../imgs/roc-auc.png)


From the confusion matrix, we can see our model is very efficient in predicting True Negatives (predict no WNV present & actual no WNV present). It also has lowest in predicting False Negative (predict no WNV present but actual WNV present) which our Recall scoring metrics (lowers False Negative) accounts for. 

Both AUC for predicting class 0 (no WNV present) and class 1 (WNV present) is high at 0.84. Based on the ROC curve, it has a good prediction at lower decision threshold of around 0.2 False Positive Rate. 

### 3.2 Feature Importance

![](../imgs/feature-selection.png)

As seen above, the top feature turned out to be `'longitude'` and `'species_culex pipiens/restuans'`, followed by `'hot_spots'`, `'year'` and `'latitude'` to round off the top 5 features. It seems like location features (`'longitude'` and `'latitude'`) were important for the model. Our engineered features `'hotspot'` and `'relativehumidity'`also fall in the top 10 features for our model.

## 4. Cost-Benefit Analysis of Spraying

To determine the cost effectiveness of spraying, we first need to consider the following:  
* Direct costs: These would mainly include the cost of procuring chemicals required for spraying.  
* Indirect costs: These would be productivity/economic costs incurred from people seeking short/long term treatment for the West Nile Virus.

**Cost of Spray**

From our Cluster Analysis, the mean average area of each hot zone cluster is **0.1576 sq km (0.44536km length and 0.3539km breath)**. 

From external research, the insecticide used to control the adult mosquitoes is [Zenivex™, and it is applied at a rate of 1.5 fluid ounces per acre.](https://abc7chicago.com/archive/9206273/)It is approved for use by the U.S. Environmental Protection Agency and is used to control mosquitoes in outdoor residential and recreational areas.

For its cost, its estimated value is approximately [USD$300 per gallon.](https://www.gfmosquito.com/wp-content/uploads/2017/07/2017-ND-Mosq.-Control-Quotes-Tabulation.pdf)

For 1.5 fluid ounce, it costs around **USD$3.52 per acre**.

For each hotzone cluster, it costs around **USD$136** per cluster, at a conversion rate of 1 acre equals 0.00404686 sq km. 

As our current model defines hotspot as 4 or more cases(i.e. 1 case per year on average), we have identified 59 hot spots. With a cost of **USD$8,024** to spray all the hot spots.

From our clustering analysis, once a cluster is sprayed, it takes approximately a week before a new WNV presence to surface again. Therefore, our recommended frequency for spraying at the hot-zones will be a week. 

As the spraying will be done on a weekly basis, we will take the period of June to August - which is approximately 13 weeks, and the total cost for the insecticides for these 13 weeks is **USD$104,312**. 

There will be additional cost for areas where a large number of mosquitoes are detected through weekly tests, and also for summers which extends deeper into September. 


**Cost of Medical Care**

About 1 in 5 people who are infected with the virus will develop a fever with other symptoms such as headache and joint pains, but about 1 in 150 of those infected develop a serious nervous system illness such as encephalitis or meningitis that typically requires hospitalization.

Mean costs attributable to WNV were USD\\$1177 (95% CI) for acute infection, USD\\$180 (95% CI) for continuing care, USD\\$11,614 (95% CI) for final care - acute death, and USD\\$3199 (95% CI) for final care - late death. Expected 1-year costs were USD\\$13,648, adjusted for survival. Three hundred seventeen infected subjects were diagnosed with at least one neurologic syndrome and greatest healthcare costs in acute infection were associated with encephalitis (USD$710, 95% CI). ([source](https://bmcinfectdis.biomedcentral.com/articles/10.1186/s12879-019-4596-9))

Using this information, a study from a research team from the Centers for Disease Control and Prevention(CDC), calculated the [costs of these subset of patients with additional related medical care and missed worked incurred in the 5 years after the initial infection](https://www.sciencedaily.com/releases/2014/02/140210184713.htm). To estimate the total cost of WNV disease to the nation, the research team extrapolated those costs to the total number of hospitalized cases of WNV disease reported to CDC since 1999. These findings suggest an annual burden of **USD56 million** in the United States.

Patients who were hospitalized with acute flaccid paralysis, a partial- to whole-body paralysis caused by WNV infection, had the largest initial and long-term medical costs. All of them required long-term care to help regain lost function, which increased costs. Patients who were hospitalized for meningitis and those hospitalized for fever incurred similar costs of initial hospitalization (median $7,500). Most meningitis and fever patients did not require long-term care.

**Cost/Benefit**

The potential medical costs and loss of lives severely outweighs the costs for spraying. Without proper spray control, the West Nile Virus could worsen, and the economic damage to the United States will increase. The benefits can be measured in terms of savings from medical expenses. [A typical household with employer health coverage spends about USD$800 a year in out-of-pocket costs](https://www.chicagobusiness.com/health-care/health-insurance-costs-surpass-20000-year-hitting-record). Benefits would also mean lesser long-term treatment for those who suffer from more serious infection, like [inflammation of the brain (encephalitis) or of the membranes surrounding the train and spinal cord (meningitis)](https://www.mayoclinic.org/diseases-conditions/west-nile-virus/symptoms-causes/syc-20350320#:~:text=In%20less%20than%201%25%20of,High%20fever).

 

## 5. Conclusion

The final model we built was successful in answering our problem statement (predicting the West Nile Virus (WNV) in mosquitoes across the city of Chicago for the missing years 2008, 2010, 2012 and 2014). With a ROC-AUC score of 0.78 from Kaggle based on unseen data, an accuracy score of 0.82, AUC score of 0.84 and Recall score of 0.72 based on holdout test sets, our model is sufficient in its prediction. Thus, we are able to generally predict areas/hotspots with potential WNV presence and with that information, can introduce potential preventive measures to cull the mosquito population and the spread of WNV in Chicago.

Furthermore, based on our cost-benefit analysis, we have deduced that the cost of spraying is not expensive and yet the spraying conducted in 2011 and 2013 in Chicago only covered 1.6% of the cluster hotspot in which we identified. Thus, the cost of spraying is not justified if the effect of spraying does not cover the heavily affected area. 

Unfortunately, this model does not directly achieve the bigger goal of eradicating the West Nile Virus from Chicago. Most of the features in the model are beyond our control, so inference would not directly help in suggesting ways to eradicate the virus. Furthermore, our model is only effective under certain weather conditions (for example hailstorms and fogs) might throw the predictions off. As mentioned, we have no control over the weather and due to climate change, certain weather patterns that may be seen in some years will not be seen in the next (for example in 2013 we found that the humidity was lower and temperature was higher than in 2011). Nevertheless, there are some aspects from our project that may assist in reducing the numbers to the brink of eliminating the virus in the future. Our findings have shown that the effect of spraying has a significant impact on reducing the number of mosquito clusters and WNV since its introduction in 2011 and we do have control over this, unlike the weather.

### 5.1 Recommendations

Based on our project, there are measures within our control that will assist in reducing the number of mosquitoes.

**Controlling the Spraying of Insecticides**  
As mentioned earlier, spraying did help to reduce clusters, especially in residential areas (with most clusters found around the edge of the area). Instead of spraying in areas without proper methods, we can focus on these three factors to improve it in terms of both cost effectiveness and efficacy:

1) Location  
Spraying can be targeted at locations where our model predicts to have a high probability of the occurrence of the virus.
2) Time of year  
Spraying efforts can be ramped up at the appropriate time of the year (July to August where higher number of mosquitoes are found) to reduce mosquito population.
3) Spray and wind direction  
Research shows that spraying along and against the wind will incur different efficacy. However, this needs further research as this can be a topic on its own.

The CDC has already implemented local [mosquito control programs](https://www.cdc.gov/mosquitoes/mosquito-control/community/what-mosquito-control-programs-do.html) which include aerial and truck spraying. A more focused method incorporating these factors will be more effective.

Other preventive measures can be introduced based on our outside research:

**Public Education**  
From our outside research, the Centers for Disease Control and Prevention (CDC) have been encouraging the general population to be vigilant and report for potential mosquito outbreaks in their area. They also have published preventive measures that the public can do to reduce the breeding of mosquitoes such as removing standing water to reduce mosquito larvae. More information are published on their [website](https://www.cdc.gov/mosquitoes/mosquito-control/community/what-mosquito-control-programs-do.html) and open for public reading which must be circulated to increase the education efforts. More can be done to help digest the information of the dangers of the West Nile Virus from their [website](https://www.cdc.gov/mosquitoes/guidelines/west-nile/introduction.html) to "bite-size" informative campaigns which can help better relay important messages to the general public.

**Increased Checks and Patrolling During Hotspot Season**  
The CDC has recommended states to employ professionals for scheduled checks for mosquito breeding in areas. Chicago could adopt this approach in order to combat the mosquito population and consequently reduce incidences of the virus. These professionals will be in charge of enforcing checks on households to ensure that there are no potential breeding grounds for mosquitoes by inspecting for stagnant/standing waters. At the same time, they can also educate the households during their rounds and spread the good practices to reduce potential mosquito clusters. In time, they may be able to reduce their regular checks if they see an improvement in such practices and a decrease in mosquito population in the area. The regularity of these checks can be further increased during seasons that have the optimal environment for mosquito breeding and WNV spreading which we found to be [75.2°F to 77°F](https://elifesciences.org/for-the-press/496ab62c/rising-temperatures-could-shift-us-west-nile-virus-transmission). By knowing these key information, professionals can better plan and prepare for a more targeted scheduled checks in the targeted areas.

**Project Wolbachia**  

There is a growing adoption of [Project Wolbachia](https://www.worldmosquitoprogram.org/en/work/wolbachia-method/how-it-works), led by the World Mosquito Program, that helps introduce the Wolbachia bacteria that occurs naturally in 50 per cent of insect species and is safe for humans and the environment. When a species of mosquitoes called 'Aedes aegypti' carry the Wolbachia, the bacteria compete with viruses like dengue, Zika, chikungunya and yellow fever. In [Singapore](https://www.straitstimes.com/singapore/health/project-wolbachia-to-be-expanded-to-cover-about-a-third-of-hdb-blocks-from-july-grace-fu) for example, it has a positive effect in suburban areas with incidence of dengue cases reduced up to 70 per cent so far in 2022. The project already has progress in different countries like Australia and Brazil. Chicago may want to adopt this project in light of their success in different parts of the countries and being spearheaded by the World Mosquito Program whose purpose is to reduce the spread of diseases from mosquito populations. More research can be done on the effect of Wolbachia on West Nile Virus as well.